<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
 

<strong>Objectives:</strong>

    - Estudiar y explorar las series correspondientes al producto 68
    - Comprobar las estacionalidades
    - Descomponer la serie
    - Obtener la media por día de la semana, mes, etc.
    - Estudiar la asignación de missings en ventas y stock
    
    
<strong>Conclusions:</strong>

   
</div>

In [1]:
import os, sys

def get_root_path(search_word):
    """ Búsqueda hacia atrás de un directorio con nombre search_word. """
    cwd = os.getcwd()
    while str(os.getcwd()).split('/')[-1] != search_word:
        os.chdir("..")
        cwd = os.getcwd()
    return cwd

ROOT_DIR = get_root_path('TFM_StockForecast')

sys.path.insert(0, ROOT_DIR) # Poner en el primer lugar del PATH (evitar paquetes con mismos nombres en otros directorios)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from src.load_data import load_csv, load_data

In [2]:
from statsmodels.tsa.seasonal import seasonal_decompose

def plot_seasonal_descompose(serie, period):
    
    result = seasonal_decompose(serie, model='additive', period=period)
    
    plt.figure(figsize=(20,40))
    plt.subplot(4,1,1)
    result.observed.plot(title='Observed')
    plt.subplot(4,1,2)
    result.trend.plot(title='Trend')
    plt.subplot(4,1,3)
    result.seasonal.plot(title='Seasonal - Period {}'.format(period))
    plt.subplot(4,1,4)
    result.resid.plot(title='Residual')
    plt.tight_layout()

In [3]:
df = load_data()
df = df.set_index("fecha")
df.head()

===================  LOAD DATASET stock ====================
Reading CSV in data/raw/03_TablaStock.csv...
=============  CLEAN CSV  ==============
Set columns to lowcase.
Sort values by date.
Dropped duplicates. Rows dropped: 628.
============  CLEAN STOCK  =============
Dropped rows with data from products with no stock data or out-of-date (descatalogados). Rows dropped: 1784
[WARNING] Number of dates missings:                                83
[WARNING] Number of dates missings (droping sundays and festivos): 63
[WARNING] Stock data with different units for same product & data. Rows: 12666
Dataset stock loaded. Shape: (25518, 3)

===================  LOAD DATASET ventas ===================
Reading CSV in data/raw/01_TablaVentas.csv...
=============  CLEAN CSV  ==============
Set columns to lowcase.
Sort values by date.
Dropped duplicates. Rows dropped: 0.
============  CLEAN VENTAS  ============
Dropped rows with data from products with no stock data or out-of-date (descatalogados). 

/mnt/c/Users/Administrador/Documents/GitHub/TFM_StockForecast/src/create_variables.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ts['missing'] = (ts.udsstock.isna()) * 1


Getting datetime variables: weekday, quarter, month, weekofyear
Getting deltaStock as the difference of Stock from today to tomorrow


/mnt/c/Users/Administrador/Documents/GitHub/TFM_StockForecast/src/create_variables.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prod_data['deltaStock'] = prod_data.udsstock.diff(periods=-1)


Getting rolling windows of last 5 days by product and weekday for column udsventa


/mnt/c/Users/Administrador/Documents/GitHub/TFM_StockForecast/src/create_variables.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_data[colname] = day_data[col].rolling(4, win_type='triang', min_periods=1).mean()


Getting rolling windows of last 5 days by product and weekday for column udsstock
Getting rolling windows of last 5 days by product and weekday for column udsprevisionempresa


,producto,udsstock,udsventa,udsprevisionempresa,promo,festivo,stockMissingType,weekday,quarter,month,weekofyear,deltaStock,roll4wd_udsventa,roll4wd_udsstock,roll4wd_udsprevisionempresa
fecha,,,,,,,,,,,,,,,
2019-06-05,1,18275.0,2833.0,102304.0,0.0,0.0,0.0,2,2,6,23,NaN,2833.0,18275.0,102304.0
2019-06-05,10,2267.0,1874.0,36888.0,1.0,0.0,0.0,2,2,6,23,NaN,1874.0,2267.0,36888.0
2019-06-05,11,2506.0,2755.0,31078.0,0.0,0.0,0.0,2,2,6,23,NaN,2755.0,2506.0,31078.0
2019-06-05,12,1279.0,1161.0,33661.0,0.0,0.0,0.0,2,2,6,23,NaN,1161.0,1279.0,33661.0
2019-06-05,13,2493.0,1603.0,32119.0,1.0,0.0,0.0,2,2,6,23,NaN,1603.0,2493.0,32119.0


# Time Series

In [ ]:
subdf = df.loc[(df.producto=='68')]
for series in ['udsventa', 'udsstock', 'udsprevisionempresa']:
    subdf[[series, 'roll5wd_'+series]].plot(figsize=(20,10), title="Producto: {}".format('68'))
    plt.show()
    


In [7]:
def _assing_missings_stock(df, col):
    """Assign missings: backfill for holiday
                        4wd rolling windows for else???
    """
    print("Assigning missings for {}".format(col))
    data = df.copy()
    holiday_data = data.loc[(data['festivo'] == 1) | (data['weekday'] == 6)]
    wd_data = data.loc[(data['festivo'] != 1) & (data['weekday'] != 6)]
    
    print("Missing in dataset:               {} ({} total rows).".format(sum(data[col].isna()), data.shape[0]))
    print("Missings in holiday days:         {} ({} total rows).".format(sum(holiday_data[col].isna()), holiday_data.shape[0]))
    print("Missings in working days:         {} ({} total rows).".format(sum(wd_data[col].isna()), wd_data.shape[0]))
    
    data['holiday_' + col] = data.fillna(method='bfill')
    print("Assigned missings for holiday data - Remaining missings:      {}".format(sum(data[col].isna())))
    
    data.loc[(data['festivo'] != 1) & (data['weekday'] != 6), col] = wd_data[col].fillna(wd_data["roll4wd_" + col])
    print("Assigned missings for working days data - Remaining missings: {}".format(sum(data[col].isna())))
    return data
col = 'udsstock'
data = subdf.copy()

# Creamos vectores de filtro
holiday = (data['festivo'] == 1) | (data['weekday'] == 6)
workday = (data['festivo'] != 1) & (data['weekday'] != 6)
missing = data[col].isna()
incrementnot0 = (~data['deltaStock'].isna()) & (data['deltaStock'] != 0)


print("Missing in dataset:               {} ({} total rows).".format(sum(missing), data.shape[0]))
print("Missings in holiday days:         {} ({} total rows).".format(sum(missing & holiday), sum(holiday)))
print("Missings in working days:         {} ({} total rows).".format(sum(missing & workday), sum(workday)))

print("In holiday days, the increment of stock was different from 0 in {} days.".format(sum(holiday & incrementnot0)))

# Creamos una columna para los holiday missings
data['holiday_missing'] = data[col].fillna(method='bfill')

# Asignamos todos los valores como rolling windows
data[col] = data[col].fillna(data["roll4wd_" + col])
print("Assigned missings for working days data - Rolling windows method.")

# Asignamos los holidays por encima y borramos la columna intermedia
data.loc[holiday & missing, col] = data.loc[holiday & missing, "holiday_missing"]
data = data.drop("holiday_missing", axis=1)
print("Assigned missings for holiday data - backfill method")
print("Assigned missings for holiday data - Remaining missings: {}".format(sum(data[col].isna())))



data


Missing in dataset:               99 (296 total rows).
Missings in holiday days:         26 (50 total rows).
Missings in working days:         73 (246 total rows).
In holiday days, the increment of stock was different from 0 in 1 days.
Assigned missings for working days data - Rolling windows method.
Assigned missings for holiday data - backfill method
Assigned missings for holiday data - Remaining missings: 5


,producto,udsstock,udsventa,udsprevisionempresa,promo,festivo,stockMissingType,weekday,quarter,month,weekofyear,deltaStock,roll4wd_udsventa,roll4wd_udsstock,roll4wd_udsprevisionempresa,deltaStock1
fecha,,,,,,,,,,,,,,,,
2019-06-05,1,18275.0,2833.0,102304.0,0.0,0.0,0.0,2,2,6,23,NaN,2833.000000,18275.00,102304.000,NaN
2019-06-06,1,NaN,17475.0,34479.0,1.0,0.0,2.0,3,2,6,23,NaN,17475.000000,NaN,34479.000,NaN
2019-06-07,1,NaN,4250.0,25127.0,0.0,0.0,2.0,4,2,6,23,NaN,4250.000000,NaN,25127.000,NaN
2019-06-08,1,NaN,NaN,98716.0,0.0,0.0,2.0,5,2,6,23,NaN,NaN,NaN,98716.000,NaN
2019-06-09,1,2920.0,NaN,NaN,0.0,0.0,2.0,6,2,6,23,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-22,1,4283.0,NaN,NaN,0.0,0.0,0.0,6,1,3,12,0.0,NaN,6696.40,NaN,-8094.0
2020-03-23,1,4283.0,NaN,2267.0,0.0,0.0,0.0,0,1,3,13,-61.0,14641.428571,6696.40,9162.500,0.0
2020-03-24,1,4344.0,NaN,3211.0,0.0,0.0,0.0,1,1,3,13,3286.0,4958.500000,10362.00,13070.750,61.0


In [5]:
subdf = df.loc[(df.producto=='1')]
subdf['deltaStock1'] = subdf.udsstock.diff(periods=1)
subdf.loc[subdf.weekday == 6]

<ipython-input-5-9d27627bcbc9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subdf['deltaStock1'] = subdf.udsstock.diff(periods=1)


,producto,udsstock,udsventa,udsprevisionempresa,promo,festivo,stockMissingType,weekday,quarter,month,weekofyear,deltaStock,roll4wd_udsventa,roll4wd_udsstock,roll4wd_udsprevisionempresa,deltaStock1
fecha,,,,,,,,,,,,,,,,
2019-06-09,1,NaN,NaN,NaN,0.0,0.0,2.0,6,2,6,23,NaN,NaN,NaN,NaN,NaN
2019-06-16,1,407.0,NaN,NaN,0.0,0.0,0.0,6,2,6,24,0.0,NaN,407.000000,NaN,-13023.0
2019-06-23,1,NaN,NaN,NaN,0.0,0.0,2.0,6,2,6,25,NaN,NaN,407.000000,NaN,NaN
2019-06-30,1,NaN,NaN,NaN,0.0,0.0,2.0,6,2,6,26,NaN,NaN,407.000000,NaN,NaN
2019-07-07,1,NaN,NaN,NaN,0.0,0.0,2.0,6,3,7,27,NaN,NaN,407.000000,NaN,NaN
2019-07-14,1,4619.0,NaN,NaN,0.0,0.0,0.0,6,3,7,28,0.0,NaN,4619.000000,NaN,NaN
2019-07-21,1,2048.0,NaN,NaN,0.0,0.0,0.0,6,3,7,29,0.0,NaN,3976.250000,NaN,-9302.0
2019-07-28,1,1725.0,NaN,NaN,0.0,0.0,0.0,6,3,7,30,0.0,NaN,3103.714286,NaN,NaN
2019-08-04,1,NaN,NaN,NaN,0.0,0.0,2.0,6,3,8,31,NaN,NaN,2276.857143,NaN,NaN


In [ ]:
plot_seasonal_descompose(subdf.udsventa, 7)

## Missings de ventas

In [ ]:
miss[['weekday','udsventa']].fillna(0).groupby("weekday")['udsventa'].count()

In [ ]:
data = subdf
col = 'roll5wd_udsventa'
miss = data.loc[data[col].isna()]
no_miss = data.loc[~data[col].isna()]
# Numero total de missings
print("Missings en la serie:                     {}".format(miss.shape[0]))
print("Missings en la serie sin cambio de stock: {}".format(miss.loc[miss.deltaStock == 0].shape[0]))
print("Missings en la serie con cambio de stock: {}".format(miss.loc[miss.deltaStock > 0].shape[0]))
print("Missings en la serie con cambio de stock: {}".format(miss.loc[miss.deltaStock < 0].shape[0]))


# Asignamos 

In [ ]:
data = subdf
col = 'udsventa'
miss = data.loc[data[col].isna()]
no_miss = data.loc[~data[col].isna()]
# Numero total de missings
print("Missings en la serie:                     {}".format(miss.shape[0]))
print("Missings en la serie sin cambio de stock: {}".format(miss.loc[miss.deltaStock == 0].shape[0]))
print("Missings en la serie con cambio de stock: {}".format(miss.loc[miss.deltaStock > 0].shape[0]))
print("Missings en la serie con cambio de stock: {}".format(miss.loc[miss.deltaStock < 0].shape[0]))

In [ ]:
no_miss.loc[no_miss.deltaStock == 0]

In [ ]:
miss.loc[miss.deltaStock == 0]

In [ ]:
subdf.loc[subdf.weekday == 6]

In [ ]:
subdf.loc[subdf.index > '2019-08-12']

## Medias de las ventas

In [ ]:
# Media de ventas por día de la semana
subdf.groupby("weekday")['udsventa'].mean()

In [ ]:
# Media de ventas por día de la semana
subdf.groupby("weekday")['udsventa'].min()

In [ ]:
# Media de ventas por día de la semana
subdf.groupby("weekday")['udsventa'].max()

In [ ]:
# Media de ventas por día de la semana
subdf.groupby("festivo")['udsventa'].min()

In [ ]:
# Media de ventas por día de la semana
subdf.groupby("festivo")['udsventa'].max()

In [ ]:
subdf

## Ventas en festivo o domingo faltantes puede ser 0?

In [ ]:
subdf = df.loc[(df.producto=='68')]
subdf.udsventa.plot(figsize=(20,10), title="Producto: {}".format('68'))
plt.show()

In [ ]:
subdf.loc[(subdf.festivo == 1) | (subdf.weekday == 6)] = subdf.loc[(subdf.festivo == 1) | (subdf.weekday == 6)].fillna(0)
subdf.udsventa.plot(figsize=(20,10), title="Producto: {}".format('68'))
plt.show()

In [ ]:
subdf.loc[(subdf.festivo == 1) | (subdf.weekday == 6)] = subdf.loc[(subdf.festivo == 1) | (subdf.weekday == 6)].fillna(0)
subdf.udsventa.plot(figsize=(20,10), title="Producto: {}".format('68'))
plt.show()

In [ ]:
subdf.loc[subdf.udsventa.isna()]